#### Set Up

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import sklearn.neighbors
import ignore warnings

# Turn off warnings
import warnings
warnings.filterwarnings('ignore')

# To save dataframe as png
import dataframe_image as dfi

In [2]:
# Read in files
stations = pd.read_csv("Resources/stations_clean.csv")
housing = pd.read_csv("Resources/geopyoutput.csv")

In [5]:
# Drop 2nd index
housing = housing.loc[:, ~housing.columns.str.contains('^Unnamed')]

# Keep relevant columns
stations = stations[['station_name', 'Daytime Routes', 'lat_field', 'lon_field', 'zipcode']] 

#### Find the closest train station to each housing record


In [6]:
# Find the absolute value of each coordinate pair
def dist(lat1, long1, lat2, long2):
    return np.abs((lat1-lat2)+(long1-long2))

In [7]:
# Extract all lat values and save to variable
lat_column = housing.loc[:,'Latitude']
lats = lat_column.values


# Extract all long values and save to variable
long_column = housing.loc[:,'Longitude']
longs = long_column.values

In [8]:
# Apply lambda function across each column and if 1 apply the function to the row
distances = stations.apply(
    lambda row: dist(lats, longs, row['lat_field'], row['lon_field']), 
    axis=1)

distances

0      [0.2798359451539909, 0.18055454192312226, 0.10...
1      [0.2692489451539899, 0.16996754192312125, 0.09...
2      [0.262133945153991, 0.16285254192312237, 0.087...
3      [0.253145945154003, 0.15386454192313437, 0.078...
4      [0.24406294515399907, 0.14478154192313042, 0.0...
                             ...                        
489    [0.25772305484600366, 0.3570044580768723, 0.43...
490    [0.278603054846009, 0.37788445807687765, 0.452...
491    [0.29267605484600523, 0.3919574580768739, 0.46...
492    [0.3223630548460008, 0.42164445807686945, 0.49...
493    [0.3086840548460046, 0.40796545807687323, 0.48...
Length: 494, dtype: object

In [9]:
# Use idxmin to calculate the closest station name

def find_station(lat, long):
    distances = stations.apply(
        lambda row: dist(lat, long, row['lat_field'], row['lon_field']), 
        axis=1)
    return stations.loc[distances.idxmin(), 'station_name']

In [10]:
# Find the closest station name to each recorded sale
closest_station = housing.apply(
    lambda row: find_station(row['Latitude'], row['Longitude']), 
    axis=1)

In [11]:
# Show closest station
closest_station

0          bay ridge-95 st
1                4 av-9 st
2                    28 st
3            greenpoint av
4              kingston av
              ...         
231    grand central-42 st
232    grand central-42 st
233    grand central-42 st
234    grand central-42 st
235                  86 st
Length: 236, dtype: object

In [12]:
# Append list of closest stations to the homes DataFrame
housing['nearest_station'] = closest_station
housing

,Street,Unit,Borough,Neighborhood,Year,Price,Postalcode,Latitude,Longitude,nearest_station
0,9602 4th Avenue,6D,Brooklyn,Bay Ridge,2019-01-01,NaN,11209,40.615187,-74.032021,bay ridge-95 st
1,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st
2,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st
3,918 Manhattan Ave,#2,Brooklyn,Greenpoint,2021-01-01,5000.0,11222,40.730751,-73.954026,greenpoint av
4,906 Prospect Place,2A,Brooklyn,Crown Heights,2021-01-01,735000.0,11213,40.673768,-73.946578,kingston av
...,...,...,...,...,...,...,...,...,...,...
231,308 East 38th Street,4C,Manhattan,Murray Hill,2018-01-01,890000.0,10016,40.746917,-73.973358,grand central-42 st
232,308 East 38th Street,8D,Manhattan,Murray Hill,2018-01-01,899000.0,10016,40.746917,-73.973358,grand central-42 st
233,308 East 38th Street,16F,Manhattan,Murray Hill,2018-01-01,1556238.0,10016,40.746917,-73.973358,grand central-42 st
234,308 East 38th Street,11E,Manhattan,Murray Hill,2018-01-01,1374723.0,10016,40.746917,-73.973358,grand central-42 st


In [14]:
stations

,station_name,Daytime Routes,lat_field,lon_field,zipcode
0,astoria-ditmars blvd,N W,40.775036,-73.912034,11101
1,astoria blvd,N W,40.770258,-73.917843,11102
2,30 av,N W,40.766779,-73.921479,11102
3,broadway,N W,40.761820,-73.925508,11101
4,36 av,N W,40.756804,-73.929575,11106
...,...,...,...,...,...
489,prince's bay,SIR,40.525507,-74.200064,10309
490,pleasant plains,SIR,40.522410,-74.217847,10309
491,richmond valley,SIR,40.519631,-74.229141,10307
492,tottenville,SIR,40.512764,-74.251961,10307


#### Merge Daytime Routes Column to Housing dataframe

In [15]:
# Left Join
df = pd.merge(housing,stations,how='left',left_on=['nearest_station'],right_on=['station_name'])


In [16]:
# Display results
df.head()

,Street,Unit,Borough,Neighborhood,Year,Price,Postalcode,Latitude,Longitude,nearest_station,station_name,Daytime Routes,lat_field,lon_field,zipcode
0,9602 4th Avenue,6D,Brooklyn,Bay Ridge,2019-01-01,NaN,11209,40.615187,-74.032021,bay ridge-95 st,bay ridge-95 st,R,40.616622,-74.030876,11209
1,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,4 av-9 st,R,40.670847,-73.988302,11209
2,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,4 av-9 st,F G,40.670272,-73.989779,11215
3,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,28 st,R W,40.745494,-73.988691,10001
4,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,28 st,1,40.747215,-73.993365,10001


#### Find the distance between two lists of geographic coordinates - Use Haversine Distance¶


In [17]:
# Convert latitude and longitude to radians and add these columns to the dataframe using np.radians

# Add columns with radians for latitude and longitude
df[['lat_radians_housing','long_radians_housing']] = (
    np.radians(df.loc[:,['Latitude','Longitude']])
)

stations[['lat_radians_stations','long_radians_stations']] = (
    np.radians(stations.loc[:,['lat_field','lon_field']])
)

In [20]:
# Add unique ID column
df['uniqueid'] = np.arange(len(df))

In [22]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (df[['lat_radians_housing','long_radians_housing']],
     stations[['lat_radians_stations','long_radians_stations']])*3959
)

# Note that 3959 is the radius of the earth in miles
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=df['uniqueid'], 
                 columns=stations['station_name'])
)

df_dist_matrix

station_name,astoria-ditmars blvd,astoria blvd,30 av,broadway,36 av,39 av-dutch kills,lexington av/59 st,5 av/59 st,57 st-7 av,49 st,...,bay terrace,great kills,eltingville,annadale,huguenot,prince's bay,pleasant plains,richmond valley,tottenville,arthur kill
uniqueid,,,,,,,,,,,,,,,,,,,,,
0,12.708664,12.271725,11.969145,11.567251,11.160972,10.843322,10.740149,10.786029,10.673340,10.309324,...,6.840540,7.666459,8.495638,9.248202,10.101492,10.779270,11.671809,12.273950,13.541639,12.962236
1,7.802755,7.355662,7.049114,6.649023,6.244883,5.929126,5.637196,5.681110,5.577430,5.218161,...,11.358216,12.114200,12.911068,13.600900,14.429260,15.138141,15.939860,16.496746,17.693960,17.134805
2,7.802755,7.355662,7.049114,6.649023,6.244883,5.929126,5.637196,5.681110,5.577430,5.218161,...,11.358216,12.114200,12.911068,13.600900,14.429260,15.138141,15.939860,16.496746,17.693960,17.134805
3,5.857789,5.477793,5.212038,4.847200,4.484732,4.206968,5.170931,5.422276,5.567393,5.358302,...,14.152509,14.960748,15.784219,16.517753,17.366504,18.060315,18.920836,19.506357,20.747836,20.175180
4,5.857789,5.477793,5.212038,4.847200,4.484732,4.206968,5.170931,5.422276,5.567393,5.358302,...,14.152509,14.960748,15.784219,16.517753,17.366504,18.060315,18.920836,19.506357,20.747836,20.175180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,12.266558,11.829444,11.526790,11.124868,10.718561,10.400889,10.302309,10.350588,10.240945,9.877840,...,7.238120,8.059570,8.887911,9.636342,10.489229,11.171186,12.057094,12.656198,13.919627,13.341127
556,12.266558,11.829444,11.526790,11.124868,10.718561,10.400889,10.302309,10.350588,10.240945,9.877840,...,7.238120,8.059570,8.887911,9.636342,10.489229,11.171186,12.057094,12.656198,13.919627,13.341127
557,12.266558,11.829444,11.526790,11.124868,10.718561,10.400889,10.302309,10.350588,10.240945,9.877840,...,7.238120,8.059570,8.887911,9.636342,10.489229,11.171186,12.057094,12.656198,13.919627,13.341127


In [23]:
# Get a series containing minimum value of each row
minValuesObj = df_dist_matrix.min(axis=1)
print('minimum value in each row : ')
print(minValuesObj)

minimum value in each row : 
uniqueid
0      0.115936
1      0.577546
2      0.577546
3      0.320612
4      0.320612
         ...   
555    0.123142
556    0.123142
557    0.123142
558    0.123142
559    0.123142
Length: 560, dtype: float64


In [25]:
# Append to housing dataframe
df['distance_miles'] = minValuesObj
df

,Street,Unit,Borough,Neighborhood,Year,Price,Postalcode,Latitude,Longitude,nearest_station,station_name,Daytime Routes,lat_field,lon_field,zipcode,lat_radians_housing,long_radians_housing,uniqueid,distance_miles
0,9602 4th Avenue,6D,Brooklyn,Bay Ridge,2019-01-01,NaN,11209,40.615187,-74.032021,bay ridge-95 st,bay ridge-95 st,R,40.616622,-74.030876,11209,0.708869,-1.292103,0,0.115936
1,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,4 av-9 st,R,40.670847,-73.988302,11209,0.710101,-1.291602,1,0.577546
2,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,4 av-9 st,F G,40.670272,-73.989779,11215,0.710101,-1.291602,2,0.577546
3,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,28 st,R W,40.745494,-73.988691,10001,0.710210,-1.290404,3,0.320612
4,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,28 st,1,40.747215,-73.993365,10001,0.710210,-1.290404,4,0.320612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,439 East 88th Street,4F,Manhattan,Upper East Side,2018-01-01,810000.0,10016,40.620910,-74.028227,86 st,86 st,N,40.592721,-73.978230,11223,0.708969,-1.292036,555,0.123142
556,439 East 88th Street,4F,Manhattan,Upper East Side,2018-01-01,810000.0,10016,40.620910,-74.028227,86 st,86 st,B C,40.785868,-73.968916,10024,0.708969,-1.292036,556,0.123142
557,439 East 88th Street,4F,Manhattan,Upper East Side,2018-01-01,810000.0,10016,40.620910,-74.028227,86 st,86 st,1,40.788644,-73.976218,10024,0.708969,-1.292036,557,0.123142
558,439 East 88th Street,4F,Manhattan,Upper East Side,2018-01-01,810000.0,10016,40.620910,-74.028227,86 st,86 st,4 5 6,40.779492,-73.955589,10037,0.708969,-1.292036,558,0.123142


#### Determine distance in miles from nearest subway station

In [35]:
# Use np.where to create Bool column --> True denotes less than 1 mile from train (lat/long in housing is zipcode based)

df['under_.25_mile'] = np.where(df['distance_miles'] <= .25, True, False)
df['under_.5_mile'] = np.where(df['distance_miles'] <= .50, True, False)
df['under_1_mile'] = np.where(df['distance_miles'] <= 1, True, False)

df.head()

<ipython-input-35-d225243401dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['under_.25_mile'] = np.where(df['distance_miles'] <= .25, True, False)
<ipython-input-35-d225243401dd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['under_.5_mile'] = np.where(df['distance_miles'] <= .50, True, False)
<ipython-input-35-d225243401dd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,uniqueid,Street,Unit,Borough,Neighborhood,Year,Price,Postalcode,Latitude,Longitude,nearest_station,Daytime Routes,distance_miles,under_.25_mile,under_.5_mile,under_1_mile
index,,,,,,,,,,,,,,,,
0,0,9602 4th Avenue,6D,Brooklyn,Bay Ridge,2019-01-01,NaN,11209,40.615187,-74.032021,bay ridge-95 st,R,0.115936,True,True,True
1,1,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,R,0.577546,False,False,True
2,2,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,F G,0.577546,False,False,True
3,3,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,R W,0.320612,False,True,True
4,4,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,1,0.320612,False,True,True


In [27]:
# Give index a name 
df.index.name = 'index'

In [28]:
df.columns

Index(['Street', 'Unit', 'Borough', 'Neighborhood', 'Year', 'Price',
       'Postalcode', 'Latitude', 'Longitude', 'nearest_station',
       'station_name', 'Daytime Routes', 'lat_field', 'lon_field', 'zipcode',
       'lat_radians_housing', 'long_radians_housing', 'uniqueid',
       'distance_miles', 'under_.25_mile', 'under_.5_mile', 'under_1_mile'],
      dtype='object')

In [30]:
# Remove irrelevant columns
df = df[['uniqueid','Street', 'Unit', 'Borough', 'Neighborhood', 'Year', 'Price',
              'Postalcode', 'Latitude', 'Longitude', 'nearest_station', 'Daytime Routes', 
              'distance_miles', 'under_.25_mile', 'under_.5_mile', 'under_1_mile']]

In [31]:
# Export image display first 5 records

sample = df.head(n=5)
dfi.export(sample, 'distance_dataframe.png')

In [33]:
df.head()

,uniqueid,Street,Unit,Borough,Neighborhood,Year,Price,Postalcode,Latitude,Longitude,nearest_station,Daytime Routes,distance_miles,under_.25_mile,under_.5_mile,under_1_mile
index,,,,,,,,,,,,,,,,
0,0,9602 4th Avenue,6D,Brooklyn,Bay Ridge,2019-01-01,NaN,11209,40.615187,-74.032021,bay ridge-95 st,R,0.115936,True,True,True
1,1,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,R,0.577546,False,False,True
2,2,94 Degraw Street,NaN,Brooklyn,Columbia Street Waterfront,2020-01-01,2350000.0,11231,40.685799,-74.003352,4 av-9 st,F G,0.577546,False,False,True
3,3,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,R W,0.320612,False,True,True
4,4,933 Lafayette Street,1st floor,Brooklyn,Bedford-Stuyvesant,2020-01-01,1095000.0,11221,40.692015,-73.934678,28 st,1,0.320612,False,True,True
